# Model Comparison

## Imports

In [21]:
# Custom
from utils.dataset_manager import fit_dataset, get_classes_weights
from utils.constant import ALL_ATTACKS, FEATURES, LABELS

# Models
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, classification_report

# Other
import warnings
from tqdm import tqdm
from joblib import dump

# Ignore warnings
warnings.filterwarnings('ignore')

## Dataset

In [3]:
n_files = 2

df_train, df_test = fit_dataset(n_files, ALL_ATTACKS, dataset_directory='./data/')

# Binary classification: Bening traffic vs. DDoS traffic
df_train = df_train[(df_train[LABELS] == ALL_ATTACKS['DDoS-SYN_Flood']) | (df_train[LABELS] == ALL_ATTACKS['BenignTraffic'])]
df_test = df_test[(df_test[LABELS] == ALL_ATTACKS['DDoS-SYN_Flood']) | (df_test[LABELS] == ALL_ATTACKS['BenignTraffic'])]

df_train[LABELS] = df_train[LABELS].apply(lambda x: 1 if x == ALL_ATTACKS['DDoS-SYN_Flood'] else 0)
df_test[LABELS] = df_test[LABELS].apply(lambda x: 1 if x == ALL_ATTACKS['DDoS-SYN_Flood'] else 0)

X_train, y_train = df_train[FEATURES], df_train[LABELS]
X_test, y_test = df_test[FEATURES], df_test[LABELS]

# Prints
print('Training Population: {}'.format(len(df_train)))
print('Testing Population: {}'.format(len(df_test)))

./data/


100%|██████████| 1/1 [00:02<00:00,  2.24s/it]


Training Population: 50774
Testing Population: 30550


## Models

Since we have unbalanced data, we have to add their weigth in each model

In [15]:
xgb_params = {
    "max_depth": [3, 4, 5, 7],
    "learning_rate": [0.1, 0.01, 0.05],
    "gamma": [0, 0.25, 1],
    "reg_lambda": [0, 1, 10],
    "scale_pos_weight": [1, 3, 5],
    "subsample": [0.8],
    "colsample_bytree": [0.5],
}

rf_params = {
    "n_estimators": [100, 200, 300],
    "max_depth": [3, 4, 5, 7],
    "min_samples_leaf": [1, 3, 5],
    "max_features": ["sqrt"],
    "class_weight": ["balanced", get_classes_weights(df_train)],
}

log_reg_params = {
    "penalty": ["l1", "l2"],
    "C": [0.1, 0.5, 1, 5, 10],
    "class_weight": ["balanced", get_classes_weights(df_train)],
}

voting_params = {
    "voting": ["soft", "hard"],
    "weights": [[1, 1, 1], [1, 2, 1], [1, 1, 2], [1, 2, 2]],
}

models = {
    'log_reg': (LogisticRegression(class_weight=get_classes_weights(df_train)), log_reg_params),
    'xgb': (XGBClassifier(), xgb_params),
    'random_forest': (RandomForestClassifier(class_weight=get_classes_weights(df_train)), rf_params),
    'voting_classifier': (VotingClassifier(estimators=[('xgb', XGBClassifier()), ('rf', RandomForestClassifier()), ('logistic', LogisticRegression())], voting='soft'), voting_params)
}

### Training

In [16]:
best_estimators = {}
for model in tqdm(models):
    print('Training {}'.format(model))
    grid = GridSearchCV(models[model][0], models[model][1], cv=5, scoring='f1_macro', n_jobs=-1)
    grid.fit(X_train, y_train)
    print('Best params: {}'.format(grid.best_params_))
    print('Best score: {}'.format(grid.best_score_))
    best_estimators[model] = grid.best_estimator_

  0%|          | 0/1 [00:00<?, ?it/s]

Training voting_classifier


4153.86s - pydevd: Sending message related to process being replaced timed-out after 5 seconds
4154.05s - pydevd: Sending message related to process being replaced timed-out after 5 seconds
4154.27s - pydevd: Sending message related to process being replaced timed-out after 5 seconds
4154.51s - pydevd: Sending message related to process being replaced timed-out after 5 seconds
/home/mdelucasg/.local/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/home/mdelucasg/.local/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:444: Conver

Best params: {'voting': 'hard', 'weights': [1, 1, 1]}
Best score: 0.9995886574038308


In [17]:
best_estimators

{'voting_classifier': VotingClassifier(estimators=[('xgb',
                               XGBClassifier(base_score=None, booster=None,
                                             callbacks=None,
                                             colsample_bylevel=None,
                                             colsample_bynode=None,
                                             colsample_bytree=None, device=None,
                                             early_stopping_rounds=None,
                                             enable_categorical=False,
                                             eval_metric=None,
                                             feature_types=None, gamma=None,
                                             grow_policy=None,
                                             importance_type=None,
                                             interaction_constraints=None,
                                             learning_ra...max_bin=None,
                        

## Evaluation

In [22]:
# Predict
best_model = None
best_score = 0
for model in tqdm(best_estimators):
    y_pred = list(best_estimators[model].predict(X_test))

    # Evaluate
    y_test = list(y_test)
    f1score = f1_score(y_pred, y_test, average='macro')
    print('Model: ', model)
    print('  accuracy_score = ', accuracy_score(y_pred, y_test))
    print('  recall_score = ', recall_score(y_pred, y_test, average='macro'))
    print('  precision_score = ', precision_score(y_pred, y_test, average='macro'))
    print('  f1_score = ', f1score)
    print('  classification_report = \n', classification_report(y_pred, y_test))

    if f1score > best_score:
        best_score = f1score
        best_model = best_estimators[model]

dump(best_model, './outputs/best_model_syn_attacks.joblib')

100%|██████████| 1/1 [00:01<00:00,  1.02s/it]

Model:  voting_classifier
  accuracy_score =  0.9996399345335516
  recall_score =  0.999197560908291
  precision_score =  0.9997147880379071
  f1_score =  0.9994559094916797
  classification_report = 
               precision    recall  f1-score   support

           0       1.00      1.00      1.00      6396
           1       1.00      1.00      1.00     24154

    accuracy                           1.00     30550
   macro avg       1.00      1.00      1.00     30550
weighted avg       1.00      1.00      1.00     30550



['./outputs/best_model.joblib']